# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

import csv
import pandas as pd
   
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'flame_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'label_map.pbtxt')

## Download Model

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [15]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'rad_change_image{}.jpg'.format(i)) for i in range(1, 181) ]
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [16]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [17]:
i=0
for image_path in TEST_IMAGE_PATHS:
    i+=1
    image = Image.open(image_path)
    image_np =load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis = 0)
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                      output_dict['detection_boxes'],
                                                      output_dict['detection_classes'],
                                                      output_dict['detection_scores'],
                                                      category_index,
                                                      instance_masks=output_dict.get('detection_masks'),
                                                      use_normalized_coordinates=True,
                                                      line_thickness=2)
    
    
    xmin = output_dict['detection_boxes'][0][0]
    ymin = output_dict['detection_boxes'][0][1]
    xmax = output_dict['detection_boxes'][0][2]
    ymax = output_dict['detection_boxes'][0][3]
    
    xmin = round(xmin*434)
    ymin = round(ymin*343)
    xmax = round(xmax*434)
    ymax = round(ymax*343)
     
    #print ('coordinates for image',i+160,'are:')
    print ((xmin+xmax)/2, (ymin+ymax)/2, max((xmax-xmin)/2,(ymax-ymin)/2))
    #print ('ymin=', ymin)
    #print ('xmax=', xmax)
    #print ('ymax=', ymax)
    #print ('x_centre', (xmin+xmax)/2)
    #print ('y_centre', (ymin+ymax)/2)
    
    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    #plt.savefig('input' + str(i)+ '.png')

217.5 172.0 76.5
217.5 172.5 79.5
215.0 171.5 80.0
215.0 172.0 80.0
216.5 172.0 79.5
216.5 172.0 79.5
216.5 172.0 79.5
215.0 169.5 81.0
216.5 171.0 79.5
217.0 170.5 79.0
216.5 169.0 79.5
215.5 171.5 80.5
219.0 170.5 78.0
214.5 171.0 79.5
216.5 170.0 78.5
215.0 172.0 80.0
217.5 171.5 79.5
216.5 173.0 79.5
216.5 168.5 79.5
216.5 171.5 79.5
215.5 170.0 80.5
213.5 172.5 79.5
216.5 171.5 79.5
217.5 170.0 78.5
216.5 171.5 79.5
216.0 171.0 80.0
216.0 169.5 80.0
216.5 173.0 79.5
216.0 172.0 80.0
216.5 170.5 79.5
216.5 170.5 79.5
216.0 170.0 80.0
215.0 172.5 80.0
215.0 172.5 80.0
216.0 169.0 80.0
215.5 173.5 79.5
214.5 172.0 79.5
216.5 171.5 79.5
215.0 168.0 80.0
215.5 171.5 80.5
216.5 168.5 79.5
216.0 173.0 79.0
217.0 173.0 79.0
215.0 173.0 80.0
216.5 171.5 79.5
219.0 171.0 78.0
219.0 171.0 78.0
215.0 169.5 80.0
216.0 170.5 81.0
216.0 170.5 81.0
216.0 172.5 81.0
216.0 171.5 81.0
214.0 173.0 81.0
217.0 173.0 80.0
215.0 172.5 81.0
213.0 172.0 81.0
218.0 172.5 80.0
214.0 174.0 81.0
214.5 171.5 81

In [18]:
'''
#for the last image in the range
length = (xmax - xmin)*434
height = (ymax - ymin)*343

print ('length=',length)
print ('height=',height)

xmin = output_dict['detection_boxes'][0][0]
ymin = output_dict['detection_boxes'][0][1]
xmax = output_dict['detection_boxes'][0][2]
ymax = output_dict['detection_boxes'][0][3]

xmin = xmin*434
ymin = ymin*343
xmax = xmax*434
ymax = ymax*343

print ('coordinates are:')
print ('xmin=', xmin)
print ('ymin=', ymin)
print ('xmax=', xmax)
print ('ymax=', ymax)
'''

"\n#for the last image in the range\nlength = (xmax - xmin)*434\nheight = (ymax - ymin)*343\n\nprint ('length=',length)\nprint ('height=',height)\n\nxmin = output_dict['detection_boxes'][0][0]\nymin = output_dict['detection_boxes'][0][1]\nxmax = output_dict['detection_boxes'][0][2]\nymax = output_dict['detection_boxes'][0][3]\n\nxmin = xmin*434\nymin = ymin*343\nxmax = xmax*434\nymax = ymax*343\n\nprint ('coordinates are:')\nprint ('xmin=', xmin)\nprint ('ymin=', ymin)\nprint ('xmax=', xmax)\nprint ('ymax=', ymax)\n"